In [1]:
import pandas as pd

raw_data = pd.read_csv("IMDB_dataset_final.csv", low_memory=False)

# printing statements to check the variables
raw_x = raw_data.review; # print(raw_x)
raw_y = raw_data.sentiment; # print(raw_y)

number_of_words_in_dic = 37500; 

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
tf.config.set_visible_devices([], 'GPU')


tokenizer = Tokenizer(num_words=number_of_words_in_dic) # to tokenize the words for learning
tokenizer.fit_on_texts(raw_x)
tokenized_sentiments = tokenizer.texts_to_sequences(raw_x) # converting the words to number arrays

vocab_size = len(tokenizer.word_index) + 1

padded_tokenized_sentiments= tf.keras.utils.pad_sequences(tokenized_sentiments, padding="post", maxlen=1000)

In [2]:
import keras
from keras import Sequential # we will be using this for the CNN
from keras import layers
import tensorflow as tf
import os

The code beneath changes every time you run it and hence shows a bad approach to extracting the embeddings

In [4]:
# importing the trained keras cnn
CNN = tf.keras.models.load_model("Trained_CNN_for_sentiment_prediction")
CNN.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1000, 50)          1875000   
                                                                 
 conv1d (Conv1D)             (None, 998, 32)           4832      
                                                                 
 global_max_pooling1d (Globa  (None, 32)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 10)                330       
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 1,880,173
Trainable params: 1,880,173
Non-trainable params: 0
______________________________________________

Getting the node embedding outputs from the STABLE CNN model

In [5]:
# getting the outputs from the embedding layer of the CNN
tf.config.set_visible_devices([], 'GPU')

embeddings = keras.Model(inputs=CNN.inputs, outputs=CNN.get_layer(name="embedding").output) 
embedding_outputs = embeddings(padded_tokenized_sentiments[0:50000:1])

print(embedding_outputs)


tf.Tensor(
[[[-0.12047946 -0.07198443 -0.03228364 ...  0.00455372 -0.14110881
   -0.00318262]
  [ 0.02522953 -0.14398038 -0.13661358 ... -0.04930472  0.06729973
   -0.17320693]
  [-0.04343206 -0.04485501  0.12286492 ... -0.04977522 -0.06138654
   -0.02183243]
  ...
  [ 0.01882174  0.00709695 -0.01237831 ... -0.06010544 -0.07130452
   -0.12374803]
  [ 0.01882174  0.00709695 -0.01237831 ... -0.06010544 -0.07130452
   -0.12374803]
  [ 0.01882174  0.00709695 -0.01237831 ... -0.06010544 -0.07130452
   -0.12374803]]

 [[-0.06885604 -0.09118391 -0.07906064 ... -0.02250706 -0.14835048
   -0.02511178]
  [-0.08446779 -0.0147468  -0.03923324 ...  0.03985319  0.07039006
   -0.01437815]
  [ 0.09084012  0.02692487  0.08453914 ...  0.00336978 -0.03451585
   -0.16510229]
  ...
  [ 0.01882174  0.00709695 -0.01237831 ... -0.06010544 -0.07130452
   -0.12374803]
  [ 0.01882174  0.00709695 -0.01237831 ... -0.06010544 -0.07130452
   -0.12374803]
  [ 0.01882174  0.00709695 -0.01237831 ... -0.06010544 -0.0713

Saving the outputs to the pandas dataframe and then a new csv

In [ ]:
import numpy as np

In [6]:
variabl = tf.Variable(embedding_outputs)
# tf.saved_model.save(variabl,"Embeddings_outputs")

In [7]:
variabl.shape

TensorShape([50000, 1000, 50])

Splitting this big tensor up into train test and validation segments

In [10]:
from sklearn import model_selection
train_x, test_x = model_selection.train_test_split(variabl, random_state=1000, test_size=0.3)

TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got array([37314, 20848, 16191, ...,  6215,  4695, 42419])